In [ ]:
# Software Name : HSLinUCB
# SPDX-FileCopyrightText: Copyright (c) 2021 Orange
# SPDX-License-Identifier: GPL-2.0
#
# This software is distributed under the GNU General Public License v2.0 license
#
# Author: David DELANDE <david.delande@orange.com> et al.

# Some orchestrator sample calls. This will set up the orchestrator, start injection, scale the deployment, retrieve context, store context in a dataset, read context from a dataset and stop injection

## import lib and configure a dataset

In [ ]:
import time
from lib.CogscalingLib import Orchestrator

sessionFile="my_new_dataset.h5" #Set the dataset to use. If the dataset does not exist and self.record == True in environment (see below) the new dataset will be created
environment = Orchestrator(debug=False, sessionFile=sessionFile)
print("wait for thread activation before sending order to kafka...")
time.sleep(3)

print("Activate debug on orchestrator thread")
status, message = environment.activateDebug(component='orchestrator')
print("message:", message)

## Environment setup and components (orchestrator, zipkin, prometheus) configuration

In [ ]:
print("Set front-dynamic-component deployment limit to min=1 and max=15")
status, message = environment.setDeploymentLimit(deployment="front-dynamic-component", min=1, max=15)
print("message:", message)

print("set prometheus thread history buffer size to 10")
status, message = environment.changePrometheusTraceNumber(trace_number=10)
print("message:", message)

print("set zipkin thread history buffer size to 100")
status, message = environment.changeZipkinTraceNumber(trace_number=100)
print("message:", message)

print("set zipkin thread lookback timewindow to 10 seconds")
status, message = environment.setZipkinLookback(lookback=12)
print("message:", message)

print("Set number of sample taken from prometheus before raising the event scaling done")
status, message = environment.setSampleNumberBeforeAcceptScaling(sampleNumber=2)
print("message:", message)

print("set zipkin service to collect by default zipkin collect only istio-ingressgateway")
status, message = environment.setZipkinService(services=['istio-ingressgateway','front-dynamic-component.default'])
print("message:", message)

print("set prometheus deployment to collect by default prometheus only collect metrics for front-dynamic-component")
status, message = environment.setPrometheusService(deployments=['front-dynamic-component'])
print("message:", message)

print("retrieve configuration")
status, message = environment.getConfig()
print("configuration:", message)

## Start and manage locust injection

In [ ]:
print("set current locust user to 5 (injection is started automatically if currently stopped)")
status, message = environment.setLocustUser(user=5,spawn_rate=1)
print("message:", message)

print("Increment locust user by 1")
status, message = environment.incrementalLocustUser(step=1,spawn_rate=1)
print("message:", message)


print("set current locust user to 10 with user creation step of 10 (injection is started automatically if currently stopped)")
status, message = environment.setLocustUser(user=10,spawn_rate=10)
print("message:", message)

## Retrieve a context from the environment

In [ ]:
print("retrieve agregated state from environment using zipkin and prometheus")
#State is returned in a unique pandas dataframe containing mix from zipkin and prometheus
#components=[{'prometheus': 'ROOT', 'zipkin': 'istio-ingressgateway'},{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])
#components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])

state = environment.getAgregatedState(components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'}],replay=False, record=False,load=10, level=1,useMetricServer = False)
print("state:", state)

print("retrieve agregated state from environment using zipkin and MetricServer")
#State is returned in a unique pandas dataframe containing mix from zipkin and prometheus
#components=[{'prometheus': 'ROOT', 'zipkin': 'istio-ingressgateway'},{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])
#components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])

state = environment.getAgregatedState(components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'}],replay=False, record=False,load=10, level=1,useMetricServer = True)
print("state:", state)

## Store and read a context in the dataset

In [ ]:
print("retrieve agregated state from environment using zipkin and prometheus and store in the dataset")
#State is returned in a unique pandas dataframe containing mix from zipkin and prometheus
#components=[{'prometheus': 'ROOT', 'zipkin': 'istio-ingressgateway'},{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])
#components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])

state = environment.getAgregatedState(components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'}],replay=False, record=True,load=10, level=1,useMetricServer = False)
print("state:", state)

print("retrieve agregated state from dataset using zipkin and prometheus")
#An internal index on each [load level,number of container] is used. On the first call the first dataset entry is retrieved. On the next call
#the second..when last index is reached, the index restarting at the beginning.
#State is returned in a unique pandas dataframe containing mix from zipkin and prometheus
#components=[{'prometheus': 'ROOT', 'zipkin': 'istio-ingressgateway'},{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])
#components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'},{'prometheus': 'back-dynamic-component', 'zipkin': 'back-dynamic-component-service.default.svc.cluster.local:80/*'}])

state = environment.getAgregatedState(components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'}],replay=True, record=False,load=10, level=1,useMetricServer = False)
print("state:", state)

## Scale the number of containers

In [ ]:
print("Scale front-dynamic-component deployment to 10 pods and wait for kubernetes completion and wait for the changes to be detected by prometheus")
status, message = environment.setKubernetesDeploymentScale(deployment="front-dynamic-component",number=10,waitKubernetes=True,waitPrometheus=True)
print("message:", message)

print("retrieve deployment status")
status, message = environment.getDeploymentState("front-dynamic-component")
print("deployment configuration:", message)

print("Increment front-dynamic-component deployment with 1 pod and wait for kubernetes completion and wait for the changes to be detected by prometheus")
status, message = environment.incrementalKubernetesDeploymentScale(deployment="front-dynamic-component",step=1,waitKubernetes=True,waitPrometheus=True)
print("message:", message)

print("Increment front-dynamic-component deployment with 1 pod and do not wait")
status, message = environment.incrementalKubernetesDeploymentScale(deployment="front-dynamic-component",step=1,waitKubernetes=False,waitPrometheus=False)
print("message:", message)

print("Scale front-dynamic-component deployment to 1 pods and wait for kubernetes completion and wait for the changes to be detected by prometheus")
status, message = environment.setKubernetesDeploymentScale(deployment="front-dynamic-component",number=1,waitKubernetes=True,waitPrometheus=True)
print("message:", message)

## Reset stats

In [ ]:
print("Reset zipkin collector stats")
status, message = environment.resetZipkinStats()
print("message:", message)

print("Reset prometheus collector stats")
status, message = environment.resetPrometheusStats()
print("message:", message)

print("reset locust stats")
status, message = environment.resetLocustStats()
print("message:", message)

## Debug

In [ ]:
print("Activate debug on orchestrator thread")
status, message = environment.activateDebug(component='orchestrator')
print("message:", message)

print("Activate debug on zipkin collector thread")
status, message = environment.activateDebug(component='zipkin')
print("message:", message)

print("Activate debug on prometheus collector thread")
status, message = environment.activateDebug(component='prometheus')
print("message:", message)

print("Activate debug on orchestrator thread")
status, message = environment.activateDebug(component='orchestrator')
print("message:", message)

print("Deactivate debug on zipkin collector thread")
status, message = environment.deactivateDebug(component='zipkin')
print("message:", message)

print("Deactivate debug on prometheus collector thread")
status, message = environment.deactivateDebug(component='prometheus')
print("message:", message)

print("Deactivate debug on orchestrator thread")
status, message = environment.deactivateDebug(component='orchestrator')
print("message:", message)

## Stop locust injection

In [ ]:

print("stop locust injection")
status, message = environment.stopLocustInjection()
print("message:", message)
